In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("../src")

In [2]:
import mesh
import pandas as pd
import numpy as np
from tqdm.autonotebook import tqdm

<ipython-input-2-0f67ae6cd0a8>:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
from nltk.corpus import wordnet as wn

In [4]:
sense_dir = mesh.get_data_dir() / "sense_data"
wn30_map = pd.read_csv(sense_dir / "cwn_wn16_wn30_map.checked.csv", encoding="UTF-8", index_col=0,
                      dtype={"cwn_id": np.object, "wn16_offset": np.object})

In [5]:
wn30_map["wn16_key"] = wn30_map.wn16_offset + wn30_map.wn16_pos
wn30_map = wn30_map.set_index("wn16_key")

## Update CWN PwnSynset

In [6]:
from CwnGraph import CwnBase

In [7]:
cwn = CwnBase()

In [8]:
niter = filter(lambda x: x[1]['node_type']==("pwn_synset"), cwn.V.items())

In [10]:
cwn.V[pwn_syn[0]]

{'node_type': 'pwn_synset', 'synset_sno': '01', 'synset_word1': 'expense'}

In [11]:
def find_wn30_map(pwn_id):
    wn16_key = pwn_id.split("_")[1].lower()
    buf = [wn16_key, wn16_key.replace("a", "s")]
    while buf:
        key = buf.pop(0)        
        if key in wn30_map.index:
            return wn30_map.loc[key].to_dict()
    return None    

In [15]:
mapped_count = 0
niter = filter(lambda x: x[1]['node_type']==("pwn_synset"), cwn.V.items())
for pwn_id, ndata in tqdm(niter):
    wn30_data = find_wn30_map(pwn_id)
    if not wn30_data:
        continue
    mapped_count += 1
    ndata.update(dict(
        wn30_name=wn30_data["wn30_syn_name"],
        wn30_lemmas=wn30_data["wn30_lemmas"],
        wn30_def=wn30_data["wn30_def"]        
    ))    
    
print(mapped_count)


4145


In [16]:
cwn.V[pwn_syn[0]]

{'node_type': 'pwn_synset',
 'synset_sno': '01',
 'synset_word1': 'expense',
 'wn30_name': 'expense.n.01',
 'wn30_lemmas': 'expense,disbursal,disbursement',
 'wn30_def': 'amounts paid for goods and services that may be currently tax deductible (as opposed to capital expenditures)'}

## Write CwnGraph data structure

In [17]:
import pickle
cwn_data_path = sense_dir/"cwn_graph_0.2.pkl"
with cwn_data_path.open("wb") as fout:
    cwn.meta = {"version": "0.2", "description": "add WN30 map to pwn_synset node (build 001)"}
    pickle.dump((cwn.V, cwn.E, cwn.meta), fout)

In [18]:
## Install new data as CWN default 
CwnBase.install_cwn(cwn_data_path)

CWN data installed


In [19]:
cwn = CwnBase()

In [20]:
cwn.V[pwn_syn[0]]

{'node_type': 'pwn_synset',
 'synset_sno': '01',
 'synset_word1': 'expense',
 'wn30_name': 'expense.n.01',
 'wn30_lemmas': 'expense,disbursal,disbursement',
 'wn30_def': 'amounts paid for goods and services that may be currently tax deductible (as opposed to capital expenditures)'}

In [21]:
cwn.find_edges(pwn_syn[0])

[<CwnRelation> generic(rev): pwn_09553033N <- 07057002,
 <CwnRelation> generic(rev): pwn_09553033N <- 03003302,
 <CwnRelation> generic(rev): pwn_09553033N <- 06526301]